In [1]:
%pip install torch torchvision numpy matplotlib


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import models
import os
from PIL import Image


In [2]:
data_dir = r"E:\GAN Output\Newly_Generated_Images"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")


In [3]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((600, 600)),  # Resize images to match original resolution
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Define dataset path
data_dir = r"E:\GAN Output\mri-based-brain-tumor-images\Brain MRI Images"

# Load dataset
train_dataset = torchvision.datasets.ImageFolder(root=r"E:\GAN Output\mri-based-brain-tumor-images\Brain MRI Images\Brain MRI Images\Train", transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=r"E:\GAN Output\mri-based-brain-tumor-images\Brain MRI Images\Brain MRI Images\Validation", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Print class names (should be ['Normal', 'Tumor'])
print(train_dataset.classes)


['Normal', 'Tumor']


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # 1 output neuron for binary classification

model = model.to(device)


C:\Users\pulki\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\pulki\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [7]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)  # Reshape labels

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training Completed!")


Epoch [1/5], Loss: 0.4125
Epoch [2/5], Loss: 0.1018
Epoch [3/5], Loss: 0.0416
Epoch [4/5], Loss: 0.0318
Epoch [5/5], Loss: 0.0394
Training Completed!


In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(images)
        predicted = torch.sigmoid(outputs) > 0.5  # Convert logits to class labels
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 97.50%


In [11]:
torch.save(model.state_dict(), "tumor_classification_resnet.pth")
print("Model Saved Successfully!")


Model Saved Successfully!
